In [1]:
import formulas
import utilities
import pandas as pd
import os
import re


def create_formulas_content(website_creator_dir, docs_dir):

    formula_file_path = website_creator_dir + os.path.sep + 'formulas.csv' 
    order_file_path = website_creator_dir + os.path.sep + 'sort_orders.csv'
    
    df_by_year =  (
        formulas.get_formulas_by_year_df(formula_file_path))
    df_cumulative = (
        formulas.get_formulas_by_year_cumulative_df(
            formula_file_path, order_file_path))
    sort_orders_df = pd.read_csv(order_file_path)
    df_formulas = pd.concat([df_by_year, df_cumulative])
    
    dirs_df = (
        df_formulas[['State', 'Formula sub category 1', 
                             'Formula sub category 2',
                             'Subject code']].drop_duplicates())
    file_paths_df = (
        df_formulas[['State', 'Formula sub category 1', 
                             'Formula sub category 2',
                             'Subject code', 'Category']].drop_duplicates())
    
    utilities.create_sub_directories_from_df(base_dir = docs_dir, 
                                             sub_paths_df = dirs_df)
    
    front_matter_index_files = {'bookCollapseSection' : True}
    utilities.create_index_files(base_dir=docs_dir, dirs_df=dirs_df, 
                       front_matter=front_matter_index_files,
                       sort_orders_df = sort_orders_df)
    
    utilities.create_files(base_dir = docs_dir, file_paths_df= file_paths_df, 
                           file_extension='.md', 
                           fn=formulas.get_formula_display_string, 
                           sort_orders_df = sort_orders_df,
                           df_formulas = df_formulas)


if __name__ == '__main__':

    website_creator_dir = os.getcwd()
    docs_dir = utilities.get_docs_path(website_creator_dir)
    utilities.delete_directory_if_it_exists(docs_dir)
    create_formulas_content(website_creator_dir, docs_dir)


    # df_calculus = formulas.df_calculus_summary(df_formulas_by_year)
    # styler_calculus = formulas.styler_calculus_summary(
    #     df_calculus, formulas.formulas_on_formula_sheet(df_formulas_by_year))
    # display(styler_calculus)

In [ ]:
# import pandas as pd
# import  os
# import utilities

# website_creator_dir = os.getcwd()
# docs_dir = utilities.get_docs_path(website_creator_dir)
# formula_file_path = website_creator_dir + os.path.sep + 'formulas.csv'
# order_file_path = website_creator_dir + os.path.sep + 'sort_orders.csv'

# formulas_df = pd.read_csv(formula_file_path)
# sort_orders_df = pd.read_csv(order_file_path)

# cumulative_hierarchy_df = sort_orders_df.copy()

# cumulative_hierarchy_df = (cumulative_hierarchy_df[
#                   (cumulative_hierarchy_df['Level_2'].str.upper() ==
#                    'BY YEAR CUMULATIVE') &
#                   (cumulative_hierarchy_df['Level_4'].isnull()) &
#                   (cumulative_hierarchy_df['Level_3'].notnull())
#                   ]).iloc[:, :4]

# cumulative_hierarchy_df = cumulative_hierarchy_df.reset_index(drop = True)
# cumulative_hierarchy_df['sort_order_hierarchy'] = cumulative_hierarchy_df.index

# subject_code_sort_df = cumulative_hierarchy_df.copy()
# subject_code_sort_df = subject_code_sort_df[['Level_3', 'sort_order_hierarchy']]

# formulas_df = pd.merge(left = formulas_df, right = subject_code_sort_df, 
#                        left_on = ['Subject code'], right_on = ['Level_3'], 
#                        how = 'left')

# formulas_df = formulas_df.rename(columns={'sort_order_hierarchy':'sort_order_formulas'})
# formulas_df = pd.merge(left = cumulative_hierarchy_df, right = formulas_df, 
#          left_on = ['Level_0'], right_on = ['State'], how = 'left')

# formulas_df = formulas_df[formulas_df['sort_order_hierarchy'] >= formulas_df['sort_order_formulas']]

# formulas_df.to_csv('test.csv')

# display(cumulative_hierarchy_df)
# display(subject_code_sort_df)
# display(formulas_df)


In [1]:
import pandas as pd
import  os
import utilities
import formulas




website_creator_dir = os.getcwd()
docs_dir = utilities.get_docs_path(website_creator_dir)
formula_file_path = website_creator_dir + os.path.sep + 'formulas.csv'
order_file_path = website_creator_dir + os.path.sep + 'sort_orders.csv'

df_by_year = formulas.get_formulas_by_year_df(formula_file_path)
df_cumulative = formulas.get_formulas_by_year_cumulative_df(formula_file_path, order_file_path)
# df.to_csv('test.csv')
display(df_by_year)
display(df_cumulative)

combined_df = pd.concat([df_by_year, df_cumulative])
display(combined_df)

,State,Formula sub category 1,Formula sub category 2,Subject code,Category,Formula_1,Formula_2
0,NSW,Formulas,By Year,Year 9,Indices,$a^m \times a^n = a^{m+n}$,NaN
1,NSW,Formulas,By Year,Year 9,Indices,$a^m \div a^n = \dfrac{a^m}{a^n} = a^{m-n}$,NaN
2,NSW,Formulas,By Year,Year 9,Indices,$(a^m)^n = a^{m \times n}$,NaN
3,NSW,Formulas,By Year,Year 9,Indices,$(ab)^n = a^nb^n$,NaN
4,NSW,Formulas,By Year,Year 9,Indices,$\left(\dfrac{a}{b}\right)^n = \dfrac{a^n}{b^n}$,NaN
...,...,...,...,...,...,...,...
59,NSW,Formulas,By Year,Year 12 Adv,Statistics,$\text{An outlier is a score with} $\n$ \text ...,NaN
60,NSW,Formulas,By Year,Year 12 Adv,Statistics,$ \text{Variance } = \sigma^2 = \dfrac{\sum(x...,NaN
61,NSW,Formulas,By Year,Year 12 Adv,Statistics,$ \text{Standard deviation } = \sqrt{\text{var...,NaN
62,NSW,Formulas,By Year,Year 9,Surds,$ \sqrt{xy} = \sqrt{x} \times \sqrt{y} $,NaN


,State,Formula sub category 1,Formula sub category 2,Subject code,Category,Formula_1,Formula_2
0,NSW,Formulas,By year cumulative,Year 9,Indices,$a^m \times a^n = a^{m+n}$,NaN
1,NSW,Formulas,By year cumulative,Year 9,Indices,$a^m \div a^n = \dfrac{a^m}{a^n} = a^{m-n}$,NaN
2,NSW,Formulas,By year cumulative,Year 9,Indices,$(a^m)^n = a^{m \times n}$,NaN
3,NSW,Formulas,By year cumulative,Year 9,Indices,$(ab)^n = a^nb^n$,NaN
4,NSW,Formulas,By year cumulative,Year 9,Indices,$\left(\dfrac{a}{b}\right)^n = \dfrac{a^n}{b^n}$,NaN
...,...,...,...,...,...,...,...
251,NSW,Formulas,By year cumulative,Year 12 Ext 1,Statistics,$\text{An outlier is a score with} $\n$ \text ...,NaN
252,NSW,Formulas,By year cumulative,Year 12 Ext 1,Statistics,$ \text{Variance } = \sigma^2 = \dfrac{\sum(x...,NaN
253,NSW,Formulas,By year cumulative,Year 12 Ext 1,Statistics,$ \text{Standard deviation } = \sqrt{\text{var...,NaN
254,NSW,Formulas,By year cumulative,Year 12 Ext 1,Surds,$ \sqrt{xy} = \sqrt{x} \times \sqrt{y} $,NaN


,State,Formula sub category 1,Formula sub category 2,Subject code,Category,Formula_1,Formula_2
0,NSW,Formulas,By Year,Year 9,Indices,$a^m \times a^n = a^{m+n}$,NaN
1,NSW,Formulas,By Year,Year 9,Indices,$a^m \div a^n = \dfrac{a^m}{a^n} = a^{m-n}$,NaN
2,NSW,Formulas,By Year,Year 9,Indices,$(a^m)^n = a^{m \times n}$,NaN
3,NSW,Formulas,By Year,Year 9,Indices,$(ab)^n = a^nb^n$,NaN
4,NSW,Formulas,By Year,Year 9,Indices,$\left(\dfrac{a}{b}\right)^n = \dfrac{a^n}{b^n}$,NaN
...,...,...,...,...,...,...,...
251,NSW,Formulas,By year cumulative,Year 12 Ext 1,Statistics,$\text{An outlier is a score with} $\n$ \text ...,NaN
252,NSW,Formulas,By year cumulative,Year 12 Ext 1,Statistics,$ \text{Variance } = \sigma^2 = \dfrac{\sum(x...,NaN
253,NSW,Formulas,By year cumulative,Year 12 Ext 1,Statistics,$ \text{Standard deviation } = \sqrt{\text{var...,NaN
254,NSW,Formulas,By year cumulative,Year 12 Ext 1,Surds,$ \sqrt{xy} = \sqrt{x} \times \sqrt{y} $,NaN
